# Flight Linker

In [1]:
from __future__ import division, print_function # because we're running Python 2
import os                                       # to make file paths independent of the operating system
import numpy as np                              # scientific computing package
import pandas as pd                             # data analysis library
import time                                     # library to handle time
import re                                       # regular expressions
import datetime as dt                           # library to handle dates
import itertools                                # iteration functions

In [ ]:
# To be replaced with user input
start_date_str = '08/04/2018'
end_date_str   = '08/04/2018'

In [2]:
def date_range(date1,date2):
    ''' Function to iterate over dates'''
    
    while date1 <= date2:
        yield date1
        date1 = date1 + dt.timedelta(days=1)
    
    
def make_datetime(row):
    ''' Function to combine date, hour and minutes into a datetime field'''
    
    time = str(int(row['TIME']))
    len_ = len(time)
    
    if len_ < 3 :
        hour   = 0
        minute = int(time)
    else:
        hour   = int(time[0:(len_-2)])
        minute = int(time[-2:])
        
    return row['FLIGHT_DATE'] + dt.timedelta(hours=hour) + dt.timedelta(minutes=minute)


def select_dates(row):
    ''' For a given service (row), select all days between start and end data
        that match the days of the week given in the frequency field.
        Return a list of all selected dates for each service.'''
    
    return [date for date in date_range(row['START'], row['END']) if 
            str(dt.datetime.weekday(date) + 1) in row['FREQ']]

def truncate_arr_date(row):
    if row['DATETIME_x'] < start_date:
        out = start_date
    else:
        out = row['DATETIME_x']
    return out

def truncate_dep_date(row):
    if row['DATETIME_y'] > end_date + dt.timedelta(days=1):
        out = end_date + dt.timedelta(days=1)
    else:
        out = row['DATETIME_y']           
    return out

In [3]:
# Create flight path compatible with all operating systems
data_path = os.path.join(os.getcwd(), 'data', '180312_ACL.csv')

# Import ACL schedule
acl_original = pd.read_csv(data_path, delimiter = ',')

print("There are %s services in this file" % (acl_original.shape[0]-1))
print("Cleaning the file...")

# Delete first row with column names
acl_services = acl_original.drop(0).reset_index(drop=True)

# Delete useless columns
acl_services.drop(['REQ','CLEAR','TERM','WKY','REC','L/N','T/R','LAST'], axis=1, inplace=True)

# Convert columns with dates to date format
acl_services['START'] = pd.to_datetime(acl_services['START'])
acl_services['END']   = pd.to_datetime(acl_services['END'])

# Create a list of weekdays of each service
acl_services['FREQ'] = [re.compile(r'\d').findall(flight) for flight in acl_services['FREQ']]

# Add column with operator
acl_services.insert(1, 'OPERATOR', [service[0:3].replace(' ','') for service in acl_services['SERVICE']])

acl_services['SERVICE'] = [x.replace(' ','') for x in acl_services.SERVICE]

There are 6045 services in this file
Cleaning the file...


In [4]:
# Initialise lists
flight_dates = []
number_dates = []

print("Working out all the flight dates of each service...")

# Create a list of all the dates that should be considered
# (apply + lambda allows to apply a calculation that does not take vectors on the entire column)
flight_dates = acl_services.apply(lambda row: select_dates(row), axis=1)

# Create a new column with the number of dates for each service
acl_services['DATE_COUNT'] = flight_dates.apply(len)

print("Repeating each service as many times as dates it flies...")

# Create a new dataframe with one row per flight, using the values in the column created above to know
# how many times to repeat each flight
acl_flights = acl_services.loc[np.repeat(acl_services.index.values, acl_services['DATE_COUNT'])]

# Turn off warning
pd.options.mode.chained_assignment = None

# Create a new column containing the flight date
acl_flights['FLIGHT_DATE'] = list(itertools.chain(*flight_dates))

# Drop columns that we don't need anymore
acl_flights.drop(['FREQ','DATE_COUNT','START','END'], axis=1, inplace=True)

Working out all the flight dates of each service...
Repeating each service as many times as dates it flies...


In [5]:
# Separate departures from arrivals
departures = acl_flights[acl_flights['A/D'] == 'D']
arrivals   = acl_flights[acl_flights['A/D'] == 'A']

# Keep only flights from one week before the stand plan week to one week after
arrivals =   arrivals[  (arrivals['FLIGHT_DATE']   >= start_date - dt.timedelta(days=7)) &
                        (arrivals['FLIGHT_DATE']   <= end_date + dt.timedelta(days=7))]

departures = departures[(departures['FLIGHT_DATE'] >= start_date - dt.timedelta(days=7)) & 
                        (departures['FLIGHT_DATE'] <= end_date + dt.timedelta(days=7))]

print("Cleaning dates and times and adding auxiliary fields...")

# Reset indeces
departures.reset_index(inplace=True, drop=True)
arrivals.reset_index  (inplace=True, drop=True)

# Create a column with date and time combining existing columns
arrivals  ['DATETIME'] = arrivals.apply  (lambda row: make_datetime(row), axis=1)
departures['DATETIME'] = departures.apply(lambda row: make_datetime(row), axis=1)

# Add a unique ID to the departures and arrivals dataframes
departures['ID']   = np.core.defchararray.add('D', departures.index.values.astype(dtype=str))

# Initialise 'LINK' field for connecting arrivals with departures
arrivals  ['LINK'] = ''
departures['LINK'] = 0

# Sort dataframes by datetime
arrivals.sort_values  ('DATETIME', inplace=True)
departures.sort_values('DATETIME', inplace=True)

# Reset indices
arrivals.reset_index  (drop=True, inplace=True)
departures.reset_index(drop=True, inplace=True)

Cleaning dates and times and adding auxiliary fields...


In [6]:
# LINKING FLIGHTS: STEP 1:
# ------------------------
# Find linking flight for all arrivals with turnaround information
# Based on the assumption that if an arriving flights contains the flight number of the turnaround,
# its departing turnaround also contains the flight number of the arrival

for arr_idx, arr in arrivals.iterrows():
    # If the turnaround field is not empty, if this arriving flight has not been assigned to a departure yet and
    # if the flight arrives inside the stand plan window
    if (arr['TROUND'] != ' '*8) & (arr['LINK'] != 1):
        
        # Subset of departures that depart after the time of the arrival and 
        # whose flight number matches the turnaround flight for this arrival
        subset = departures[(departures['SERVICE'] == arr['TROUND']) &
                            (departures['TROUND']  == arr['SERVICE'])&
                            (departures['DATETIME'] > arr['DATETIME'])]
        
        if len(subset['DATETIME']) != 0:
            # Get the index of the departing flight with the earliest time
            dep_idx = subset['DATETIME'].idxmin()
            arrivals.loc  [arr_idx,'LINK'] = departures.iloc[dep_idx]['ID']
            departures.loc[dep_idx,'LINK'] = 1

In [7]:
# LINKING FLIGHTS: STEP 2:
# ------------------------
# Find linking flight for all arrivals without turnaround information

for arr_idx, arr in arrivals.iterrows():
    
    # If the turnaround field is not empty and if the flight departs inside the stand plan window
    if arr['LINK'] == '':
        # Subset of arrivals candidates
        subset = departures[(departures['AC']       == arr['AC'])    &    # must have the same aircraft
                            (departures['OPERATOR'] == arr['OPERATOR']) & # operated by the same airline                           
                            (departures['DATETIME'] >= arr['DATETIME'] + dt.timedelta(minutes=20)) &
                            (departures['LINK']     == 0)]               # cannot be linked to another dep
        
        #if len(subset['DATETIME']) == 0: pdb.set_trace()
        
        if len(subset['DATETIME']) != 0:
            # Get the index of the departing flight with the earliest time
            dep_idx = subset['DATETIME'].idxmin() 
            arrivals.loc  [arr_idx,'LINK'] = departures.iloc[dep_idx]['ID']
            departures.loc[dep_idx,'LINK'] = 1

In [8]:
arrivals.drop  (['TIME'], axis=1, inplace=True)
departures.drop(['TIME'], axis=1, inplace=True)

In [9]:
# Perform inner join of departures and arrivals
linked = arrivals.merge(departures, left_on='LINK', right_on='ID', how='inner')

# Add field with turnaround time
linked['TURN_MINUTES'] = [delta.seconds//60 for delta in (linked.DATETIME_y - linked.DATETIME_x)]

In [10]:
# Sparse dataset for optimization
linked_clean = linked.copy()

# Filter flights in stand plan week
linked_clean = linked_clean[(linked_clean['FLIGHT_DATE_x'] <= end_date) & \
                            (linked_clean['FLIGHT_DATE_y'] >= start_date)]

linked_clean.reset_index(drop=True, inplace=True)

# Eliminate irrelevant fields
linked_clean.drop(['LINK_x','A/D_x','A/D_y','AC_y','TROUND_y','ID','LINK_y',\
                   'OPERATOR_y', 'FLIGHT_DATE_x','FLIGHT_DATE_y'], axis=1, inplace=True)

print("%s flights were linked" % (linked_clean.shape[0]))

# If the flight datetimes are outside the allocation window, truncate them
linked_clean.DATETIME_x = linked_clean.apply(lambda row: truncate_arr_date(row), axis=1)
linked_clean.DATETIME_y = linked_clean.apply(lambda row: truncate_dep_date(row), axis=1)

234 flights were linked


In [14]:
# Perform outer join for analysis of flights that were not linked
linked_outer = arrivals.merge(departures, left_on='LINK', right_on='ID', how='outer')

In [15]:
# Export files
data_path = os.path.join(os.getcwd(), 'output', 'arrivals.csv')
arrivals.    to_csv(data_path, index=False)

data_path = os.path.join(os.getcwd(), 'output', 'departures.csv')
departures.  to_csv(data_path, index=False)

data_path = os.path.join(os.getcwd(), 'output', 'turnarounds.csv')
linked_clean.to_csv(data_path, index=False)

data_path = os.path.join(os.getcwd(), 'output', 'turns_outer.csv')
linked_outer.to_csv(data_path, index=False)